In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



for root, dirs, files in os.walk("/kaggle/input"):
    print(root)



# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/metadata.pkl
/kaggle/input/faiss.index
/kaggle/input


In [2]:
!pip install faiss-cpu --no-cache-dir

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 189.4 MB/s eta 0:00:00a 0:00:01


In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch



In [3]:
# ─── Cell 1: Imports & load index ─────────────────────────────────────────────
import faiss
import pickle
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

# 1) Load FAISS index & metadata
index = faiss.read_index("/kaggle/input/faiss.index")
with open("/kaggle/input/metadata.pkl", "rb") as f:
    df = pickle.load(f)

# 2) Init your embedder
model = SentenceTransformer("all-MiniLM-L6-v2")


2025-12-11 09:50:00.222516: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765446600.404393      38 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765446600.468532      38 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
# ─── Cell 2: Pure dense‐retrieval search function ─────────────────────────────
def search_titles_urls(query: str, k: int = 5) -> pd.DataFrame:
    """
    Returns the top‐k (title, url) for the given query using
    MiniLM embeddings + FAISS inner‐product search.
    """
    qv = model.encode([query]).astype("float32")
    D, I = index.search(qv, k)

    # grab & dedupe
    res = (
        df.iloc[I[0]][["title","url"]]
        .drop_duplicates()
        .reset_index(drop=True)
    )
    return res


In [5]:

from IPython.display import HTML, display

# 1) Run your search as before
query   = "agentic ai"
results = search_titles_urls(query, k=10)

# 2) Convert the 'url' column into clickable links
results_html = results.to_html(
    escape=False,
    index=False,
    formatters={
        "url": lambda x: f'<a href="{x}" target="_blank">{x}</a>'
    }
)

# 3) Display as HTML
display(HTML(results_html))


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

title,url
"Agentic AI. Hello everyone, and welcome! We’ve seen… | by Servifyspheresolutions | Jun, 2025 | Medium",https://medium.com/@servifyspheresolutions/agentic-ai-658024c6c4c8
"Top 5 Agentic AI Frameworks to Watch in 2025 | by Future AGI | Jun, 2025 | Medium",https://medium.com/@future_agi/top-5-agentic-ai-frameworks-to-watch-in-2025-9573c09da488
"Agent VII — Navigating the Agentic AI Landscape: A Deep Dive into Frameworks Powering Next-Gen AI Agents | by DhanushKumar | May, 2025 | Medium",https://medium.com/@danushidk507/agent-vii-navigating-the-agentic-ai-landscape-a-deep-dive-into-frameworks-powering-next-gen-ai-c3b18102408e
How to Build Agentic AI: A Comprehensive Guide 🧠 | GCS Network,https://globalcybersecuritynetwork.com/blog/how-to-build-agentic-ai/
Getting Started Guide to Agentic AI | Chai,https://www.chaione.com/start
Agentic AI is in the aiR | Relativity Blog,https://www.relativity.com/blog/agentic-ai-is-in-the-air/
"Agentic AI: A Technical Deep Dive Toward Next-Generation Intelligence | by Adnan Masood, PhD. | Medium",https://medium.com/@adnanmasood/agentic-ai-a-technical-deep-dive-toward-next-generation-intelligence-140b35b71ce5
​Agentic AI in 2025: A Deep Dive into Autonomous Intelligence and Its Future |QualityPoint Technologies (QPT),https://www.blog.qualitypointtech.com/2025/04/agentic-ai-in-2025-deep-dive-into.html
Agentic AI Architectures And Design Patterns | by Anil Jain | AI / ML Architect | Data Architect | Medium,https://medium.com/@anil.jain.baba/agentic-ai-architectures-and-design-patterns-288ac589179a


In [7]:
# Cell: Load HF summarization model (lightweight)
MODEL_NAME = "google/flan-t5-base"   # light, instruction-tuned, good balance

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

tokenizer_summ = AutoTokenizer.from_pretrained(MODEL_NAME)
model_summ = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME).to(device)
model_summ.eval()


Using device: cuda


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [8]:
# Cell: helper to retrieve chunk text from df row (tries common column names)
def get_chunk_text_from_row(row):
    # try common column names (adapt if your df uses something else)
    possible_cols = ['text','chunk_text','content','body','clean_text','article','chunk']
    for c in possible_cols:
        if c in row.index and isinstance(row[c], str) and len(row[c].strip())>0:
            return row[c]
    # Fallback: if none found, try combining title + url (still better than nothing)
    title = row['title'] if 'title' in row.index else ''
    url = row['url'] if 'url' in row.index else ''
    return f"{title}\n{url}"


In [9]:
# Cell: summarizer function
def summarize_text(text, max_tokens=80):
    # keep input length reasonable for FLAN-T5; trim very long chunks
    max_input_chars = 3000
    if len(text) > max_input_chars:
        text = text[:max_input_chars]  # simple truncation; can be improved

    prompt = f"Summarize the following text in 2-3 short sentences:\n\n{text}"
    inputs = tokenizer_summ(prompt, truncation=True, return_tensors="pt").to(device)

    # generation params - short summary, deterministic beams
    out = model_summ.generate(
        **inputs,
        max_new_tokens=max_tokens,
        num_beams=4,
        early_stopping=True,
        no_repeat_ngram_size=3
    )
    summary = tokenizer_summ.decode(out[0], skip_special_tokens=True)
    return summary


In [10]:
# Cell: search + summarization - minimal changes to existing pipeline
def search_with_summaries(query: str, k: int = 5, summarise_each=True):
    """
    Returns a DataFrame with columns: title, summary (if summarise_each), url
    Uses the existing SentenceTransformer embedder (named `model` in your notebook),
    the FAISS index (named `index`), and the metadata df (named `df`).
    """
    # 1) encode query (your existing model variable from sentence-transformers)
    qv = model.encode([query]).astype("float32")   # model is SentenceTransformer
    D, I = index.search(qv, k)                     # I is shape (1,k)
    idxs = I[0].tolist()

    # 2) build result df from the metadata df rows
    results = df.iloc[idxs].copy().reset_index(drop=True)

    # 3) optionally summarize each chunk
    if summarise_each:
        summaries = []
        for i, row in results.iterrows():
            txt = get_chunk_text_from_row(row)
            try:
                summ = summarize_text(txt, max_tokens=70)   # ~3 short sentences
            except Exception as e:
                summ = "Summary generation failed"
                print("summarize error:", e)
            summaries.append(summ)
        results['summary'] = summaries
    else:
        results['summary'] = ""

    # keep only the fields we want to show
    show_cols = []
    if 'title' in results.columns: show_cols.append('title')
    show_cols.append('summary')
    if 'url' in results.columns: show_cols.append('url')

    return results[show_cols]


In [11]:
# Cell: run search_with_summaries and show clickable links
query = "Attention mechanisms"   # or input from your UI
results = search_with_summaries(query, k=10, summarise_each=True)

# make url clickable in HTML
results_html = results.to_html(
    escape=False,
    index=False,
    formatters={
        'url': lambda x: f'<a href="{x}" target="_blank">{x}</a>'
    }
)
from IPython.display import HTML, display
display(HTML(results_html))


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

title,summary,url
What is an attention mechanism? | IBM,An attention mechanism is a machine learning technique that directs deep learning models to prioritize (or attend to) the most relevant parts of input data.,https://www.ibm.com/think/topics/attention-mechanism
Attention and Memory in Deep Learning and NLP · Denny's Blog,Attention Mechanisms in Neural Networks are loosely based on the visual attention mechanism found in humans.,https://dennybritz.com/posts/wildml/attention-and-memory-in-deep-learning-and-nlp/
The Mathematics of Attention Mechanisms: Building Self-Attention from First Principles | by Abduldattijo | Generative AI,"The Mathematics of Attention Mechanisms: Building Self-Attention from First Principles | by Abduldattijo | Generative AI Sitemap Open in app Sign up Sign in Medium Logo Write Sign in Genreative AI Follow publication Stay updated with the latest news, research, and developments in the world of generative AI. We",https://generativeai.pub/the-mathematics-of-attention-mechanisms-building-self-attention-from-first-principles-b9895e9ee491
5 Attention Mechanism Insights Every AI Developer Should Know,"AI Education October 1, 2024 Written by Tobias Jaeckel Unstructured Data Management Platform » AI Education » Attention Mechanism: Everything You Need to Know",https://shelf.io/blog/attention-mechanism/
5 Attention Mechanism Insights Every AI Developer Should Know,"AI Education October 1, 2024 Written by Tobias Jaeckel Unstructured Data Management Platform » AI Education » Attention Mechanism: Everything You Need to Know",https://shelf.io/blog/attention-mechanism/
Rethinking Thinking: How Do Attention Mechanisms Actually Work? | Towards Data Science,Rethinking Thinking: How Do Attention Mechanisms Actually Work?,https://towardsdatascience.com/rethinking-thinking-how-do-attention-mechanisms-actually-work-a6f67d313f99/
Day 58: Attention Mechanisms — Foundation of Transformer Models | by Adithya Prasad Pandelu | Medium,Adithya Prasad Pandelu explores the foundation of modern AI models.,https://medium.com/@bhatadithya54764118/day-58-attention-mechanisms-foundation-of-transformer-models-c3139d3c0e79
Attention Mechanism in Deep Learning : Simplified | by Prakhar Ganesh | Medium,"In an attempt to borrow inspiration from how a human mind works, researchers in Deep Learning have tried replicating this behavior using what is known as the ‘attention mechanism’",https://medium.com/@prakhargannu/attention-mechanism-in-deep-learning-simplified-d6a5830a079d
"A study on Attention mechanism. This study shows the impact, evolution… | by Nilesh Barla | PerceptronAI | Medium",Attention has become the core operation in modern deep learning models. Today most of the commercial large models use the attention mechanism in their architecture whether it be vision or language.,https://medium.com/perceptronai/a-study-on-attention-mechanism-7d199cf783b6
Demystifying Attention Mechanisms: Self-Attention from Math to Code -I | by M K Pavan Kumar | Stackademic,"Stackademic is a learning hub for programmers, devs, coders, and engineers. Our goal is to democratize free coding education for the world.",https://blog.stackademic.com/demystifying-attention-mechanisms-self-attention-from-math-to-code-i-a935647f337c
